# Matrix Factorization Collaborative Filtering Recommendation 

This notebook will showcase how a Recommender System is built with one type of Collaborative Filtering techniques, which models the users' interests by collecting other users' preferences(collaborating) to provide personalized recommendations(filtering). The Collaborative Filtering techniques are commonly devided into two types: Model-based and Memory-based.<br>

<br>

**Model-based CF:**<br>
The prediction relies on a offline learned model that is re-trained (updated) periodically. There are various of techniques proposed in recent years to feed in Collaborative Filtering Recommender System, such as Matrix Factorization, Association Rules mining, Probailistic models, and deep neural networks. Matrix Factorization is the technique that I will showcase in this notebook.<br>

- What is **Matrix Factorization**?<br>

Given user's preferences or attitudes from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items. Matrix factorization can be done by various methods including Support Vecot Decomposition (SVD), Probabilistic Matrix Factorization (PMF), and Non-Negative Matrix Factorization (NMF).

![mf](images/Matrix_factorization.PNG)
<br>
Singular Vector Decomposition (SVD) is a powerful dimensionality reduction technique that derives the tastes and preferences of users from the raw data.The goal is to learn the latent preferences of users and the latent attributes of items from known ratings (learn features that describe the characteristics of ratings) to then predict the unknown ratings through the dot product of the latent features of users and items.
<br>
<br>
**Memory-based CF:**<br>
The rating matrix is directly used to calculate user/item similarity and make rating predictions. More information about this type of Collaborating Filtering will be provided in [another notebook](https://github.com/Olliang/All-About-Movie-Data/blob/master/Memory_Based_Collaborative_Filtering.ipynb) that showcase Memory-based Recommender System building.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')

In [2]:
# we will use MovieLens data for this model
ratings = pd.read_csv('ratings.dat', sep='::', 
                      header = None, 
                      names = ['user_id', 'movie_id', 'rating', 'timestamp'])
users = pd.read_csv('users.dat', sep='::', 
                    header = None, 
                    names = ['user_id', 'gender', 'age', 'occu_id', 'zipcode'])
movies = pd.read_csv('movies.dat', sep='::', 
                    header = None, 
                    names = ['movie_id', 'title', 'genres'])

In [5]:
occupation = {0:  "other", 1:  "academic/educator", 2:  "artist", 3:  "clerical/admin", 4:  "college/grad student",5:  "customer service",
 6:  "doctor/health care", 7:  "executive/managerial", 8:  "farmer", 9:  "homemaker", 10:  "K-12 student", 11:  "lawyer",
 12:  "programmer", 13:  "retired", 14:  "sales/marketing", 15:  "scientist", 16:  "self-employed", 17:  "technician/engineer",
 18:  "tradesman/craftsman", 19:  "unemployed", 20:  "writer"}
occupation = pd.DataFrame({'occu_id': list(occupation.keys()), 'occupation':list(occupation.values())})
occupation.head()

,occu_id,occupation
0,0,other
1,1,academic/educator
2,2,artist
3,3,clerical/admin
4,4,college/grad student


In [6]:
# Some dimensions:
n_users = users.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('The number of users: ',n_users)
print('-------------')
print('The number of movies: ',n_movies)

The number of users:  6040
-------------
The number of movies:  3706


In [7]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.iloc[:5,:5]

movie_id,1,2,3,4,5
user_id,,,,,
1,5.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0


In [51]:
# The sparsity level of the data
sparsity = round(1.0 - len(ratings) / float(n_users * n_movies), 3)
print('The sparsity level of MovieLens1M dataset is ',  str(sparsity * 100) ,'%')

The sparsity level of MovieLens1M dataset is  95.5 %


## Support Vector Decomposition (SVD)

This technique works essentially with 4 matrixes and their relationship is shown as the graph I drawn below: <br>
![svd](images/svd_graph.PNG)
<br>

**R:** input rating matrix with m users as index and n movies as columns (m x n)<br>
**U:** pofiles of users - latent features of users extracted by a dimensionality reduction technique(PCA). (m x m)<br>
**Sigma:** a simple diagonal matrix that only serves the scale of values we end up with into the proper scale. (m x n)<br>
**V:** profiles of movies- latent features of movies extracted by a dimensionality reduction technique(PCA). (n x n)<br>
<br>

- *The idea of SVD:* <br>
All this SVD does is running PCA on both the users and the items, and then giving us back that matrix that is the factors of the rating matrix. In other words, by decomposing R matrix into these 3 matrixes, we can use U and V to reconstruct an R matrix that fills in the ratings missing for any combination of users and items. This technique is a built-in algorithm that was used widely during the Netflix price amongst the leaders in the competition.<br>


- *How can we reconstruct a R matrix with high sparsity?*<br>
We will use SVD to restructure the user-item matrix into low-rank structure that keep only top k features, which represents the matrix by the multiplication of two low-rank matrices. This generated matrix can be fit to approximate the original matrix but with all the missing ratings filled.


### Modeling SVD Mannually

In [11]:
R = Ratings.as_matrix()
user_rating_mean=  np.mean(R, axis =1) # the mean of each row are all put into an array
Rating_standardized = R - user_rating_mean.reshape(-1,1) 

In [12]:
# Set up SVD
from scipy.sparse.linalg import svds
U, sigma, V = svds(Rating_standardized, k = 50)
# Convert the sigma values into diagonal matrix format
sigma = np.diag(sigma)

### Making Predictions

In [13]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), V) + user_rating_mean.reshape(-1, 1)
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [14]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (userID)]
    user_full = (user_data.merge(movies, how = 'left', left_on = 'movie_id', right_on = 'movie_id').
                     sort_values(['rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies[~movies['movie_id'].isin(user_full['movie_id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'movie_id',
               right_on = 'movie_id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [15]:
already_rated, predictions = recommend_movies(preds, 1310, movies, ratings, 20)

User 1310 has already rated 24 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [16]:
# Top 20 movies that User 1310 has rated 
already_rated.head(20)

,user_id,movie_id,rating,timestamp,title,genres
5,1310,2248,5,974781573,Say Anything... (1989),Comedy|Drama|Romance
6,1310,2620,5,974781573,This Is My Father (1998),Drama|Romance
7,1310,3683,5,974781935,Blood Simple (1984),Drama|Film-Noir
15,1310,1704,5,974781573,Good Will Hunting (1997),Drama
1,1310,1293,5,974781839,Gandhi (1982),Drama
12,1310,3101,4,974781573,Fatal Attraction (1987),Thriller
11,1310,1343,4,974781534,Cape Fear (1991),Thriller
20,1310,2000,4,974781892,Lethal Weapon (1987),Action|Comedy|Crime|Drama
18,1310,3526,4,974781892,Parenthood (1989),Comedy|Drama
17,1310,3360,4,974781935,Hoosiers (1986),Drama


In [17]:
# Top 20 movies that User 1310 hopefully will enjoy
predictions

,movie_id,title,genres
1618,1674,Witness (1985),Drama|Romance|Thriller
1880,1961,Rain Man (1988),Drama
1187,1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
1216,1242,Glory (1989),Action|Drama|War
1202,1225,Amadeus (1984),Drama
1273,1302,Field of Dreams (1989),Drama
1220,1246,Dead Poets Society (1989),Drama
1881,1962,Driving Miss Daisy (1989),Drama
1877,1957,Chariots of Fire (1981),Drama
1938,2020,Dangerous Liaisons (1988),Drama|Romance


These look like pretty descent recommendations, because it recommendes mostly drama/romance movies for the users who have mostly watched and rated highly on this genre.

## Modeling and Evaluation with Surprise package


We can either mannually calculate the rmse as how we did in the [memory-based CF notebook](https://github.com/Olliang/All-About-Movie-Data/blob/master/Memory_Based_Collaborative_Filtering.ipynb), or use a powerful `surprise` package to evaluate its RMSE using cross-validation. Among various types of cross validation, I will showcase three methods to compare for your reference.


In [ ]:
# Import libraries from Surprise package
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate

# Load Reader library
reader = Reader()

# Note: The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)
# define SVD algorithm
algo = SVD()

- **Cross validation**<br>
A normal cross validation can be done in a simple one line using Surprise.

In [19]:
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8742  0.8747  0.8743  0.8720  0.8749  0.8740  0.0011  
MAE (testset)     0.6860  0.6872  0.6866  0.6841  0.6864  0.6861  0.0011  
Fit time          38.84   39.00   39.73   39.69   42.17   39.88   1.20    
Test time         1.60    1.53    1.66    1.60    1.85    1.65    0.11    


{'test_rmse': array([0.87418729, 0.8747476 , 0.87428834, 0.87196658, 0.8748507 ]),
 'test_mae': array([0.68596547, 0.68716491, 0.68664592, 0.68406201, 0.68644151]),
 'fit_time': (38.84155821800232,
  38.99787998199463,
  39.72665524482727,
  39.688095569610596,
  42.16882634162903),
 'test_time': (1.595324993133545,
  1.5303189754486084,
  1.661365032196045,
  1.6018240451812744,
  1.8498263359069824)}

- **Cross validation Iterator**<br>
For a better control, we can also instanciate a cross-validation iterator, and make predictions over each split using the `split()` method of the iterator, and the `test()` method of the algorithm. Here is an example where we use a classical K-fold cross-validation procedure with 3 splits:

In [22]:
from surprise import accuracy
from surprise.model_selection import KFold

# define a cross-validation iterator
kf = KFold(n_splits=4)

for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.8797
RMSE: 0.8775
RMSE: 0.8775
RMSE: 0.8777


- **GridSearch Cross validation**<br>
Surprise can do much more to prevent overfiting such as GridSearchCV. The `cross_validate()` function reports accuracy metric over a cross validation procedure for a given set of parameters. If you want to know which parameter combination yields the best results, the `GridSearchCV` class comes to the rescue. Given a dictionary of parameters, this class exhaustively tries all the combinations of parameters and reports the best parameters for any accuracy measure (averaged over a different splits). <br>
For more information and code examples you can check on the [documentation](https://surprise.readthedocs.io/en/stable/getting_started.html#tune-algorithm-parameters-with-gridsearchcv).

In [24]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [40, 50, 60, 80, 100], 'n_epochs': [5, 10,20], 'lr_all': [0.002, 0.005],
              'reg_all': [0.4, 0.6]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.9295493505742103
{'n_factors': 40, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}


From the evaluation result above, we can see the improve from normal cross validation to GridSearchCV. We will use the selected hyperparameters recommended from GridSearchCV to inspect more details about the predictions.

In [27]:
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=0.2)
final_algo = SVD(n_factors = 40, n_epochs =10, lr_all= 0.005, reg_all=0.4)
predictions = final_algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.9290


0.9289939072978244

In [29]:
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [30]:
df.head(10)

,uid,iid,rui,est,details,Iu,Ui,err
0,3258,923,5.0,4.088225,{'was_impossible': False},34,885,0.911775
1,1120,2993,4.0,3.799285,{'was_impossible': False},590,321,0.200715
2,5543,1079,4.0,3.809714,{'was_impossible': False},271,1060,0.190286
3,1556,1665,4.0,2.709078,{'was_impossible': False},265,155,1.290922
4,3615,3814,5.0,3.614742,{'was_impossible': False},132,175,1.385258
5,1268,1244,5.0,4.235781,{'was_impossible': False},163,591,0.764219
6,3487,1202,5.0,3.392365,{'was_impossible': False},253,60,1.607635
7,1880,2407,4.0,3.357352,{'was_impossible': False},765,814,0.642648
8,3879,3639,3.0,3.536953,{'was_impossible': False},36,368,0.536953
9,1922,2094,4.0,3.173655,{'was_impossible': False},338,577,0.826345


**How to interpret?** <br>
Each row contains an estimated rating for a particular item from a particular user along with other information:<br>
<br>
Iu: the number of items this particular user has rated <br>
Ui: the number of users has rated the particular item <br>
rui: actual rating score<br>
est: predicted rating score<br>
err: absolute error between the actual rating and predicted rating<br>
<br>
We can sort the predictions by the absolute error of each of them to inspect the best and worst 10 predictions.

In [31]:
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [32]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
100796,101,1198,5.0,5.000000,{'was_impossible': False},85,2017,0.000000
132143,3902,1198,5.0,5.000000,{'was_impossible': False},129,2017,0.000000
6349,3902,1953,5.0,5.000000,{'was_impossible': False},129,686,0.000000
30426,2155,318,5.0,5.000000,{'was_impossible': False},60,1751,0.000000
73906,101,260,5.0,5.000000,{'was_impossible': False},85,2384,0.000000
165285,4801,2571,5.0,5.000000,{'was_impossible': False},156,2088,0.000000
26978,3272,2795,4.0,4.000001,{'was_impossible': False},652,448,0.000001
110125,5264,898,4.0,4.000008,{'was_impossible': False},129,461,0.000008
134841,1536,2797,4.0,3.999991,{'was_impossible': False},27,1192,0.000009
80162,1727,3494,4.0,4.000015,{'was_impossible': False},417,258,0.000015


In [33]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
91874,1847,1196,1.0,4.440066,{'was_impossible': False},35,2377,3.440066
28295,3919,2858,1.0,4.448828,{'was_impossible': False},57,2732,3.448828
166826,2368,296,1.0,4.451925,{'was_impossible': False},184,1713,3.451925
52913,5270,858,1.0,4.453682,{'was_impossible': False},88,1769,3.453682
89278,3715,750,1.0,4.456868,{'was_impossible': False},255,1063,3.456868
26535,1111,1193,1.0,4.478355,{'was_impossible': False},25,1351,3.478355
149703,3558,1294,1.0,4.507150,{'was_impossible': False},239,869,3.507150
121267,6013,50,1.0,4.600319,{'was_impossible': False},90,1425,3.600319
113681,3902,2022,1.0,4.672079,{'was_impossible': False},129,203,3.672079
70529,4801,1374,1.0,4.704734,{'was_impossible': False},156,1161,3.704734
